In [1]:
import sys
print(sys.executable)

C:\Users\derrjohn\OneDrive - Qualcomm\Desktop\git\env_onnxruntime\Scripts\python.exe


In [2]:
# Let's check to ensure we are in the proper environment, remember we need to be in pure arm64
import platform

arch = platform.machine()
sys = platform.system()
processor = platform.processor()
print(f"{arch}\n{sys}\n{processor}")

AMD64
Windows
AMD64 Family 25 Model 116 Stepping 1, AuthenticAMD


In [48]:
# Necessary tools that we need
import onnxruntime as ort
import os
import numpy as np
import time

from pathlib import Path
from tokenizers import Tokenizer

In [4]:
# # Preparing inputs for prompt
# batch_size = 1
# seq_len = embedding_output.shape[1]
# hidden_size = embedding_output.shape[2]
# num_heads = 2
# attn_head_size = 128 #hidden_size // num_heads
# num_layers = 28
# max_seq_len = 64
# temp = 0.6

In [5]:
# Grab the root directory as a reference
root_dir = Path.cwd().parent.parent
root_dir

WindowsPath('C:/Users/derrjohn/OneDrive - Qualcomm/Desktop/git/qnn_sample_apps')

In [6]:
# Grab the path to onnxruntime therefore we can grab hexagon driver
onnx_root = Path(ort.__file__).parent
onnx_root

WindowsPath('C:/Users/derrjohn/OneDrive - Qualcomm/Desktop/git/env_onnxruntime/Lib/site-packages/onnxruntime')

In [7]:
# Subdirectory where all .onnx dependencies are located
model_subdirectory = "cpu-deepseek-r1-distill-qwen-7b"

# The embeddings model is entry point, use netron to visualize
model_name = "deepseek-r1-distill-qwen-7b-cpu-int4-rtn-block-32-acc-level-4.onnx"

# Genai configuration path
configuration_json = "genai_config.json"

# Tokenizer
tokenizer_json = "tokenizer.json"

In [8]:
# Solidifying all paths

model_path = root_dir/"models"/model_subdirectory/model_name
tokenizer_path = root_dir/"models"/model_subdirectory/tokenizer_json
config_path = root_dir/"models"/model_subdirectory/configuration_json

In [9]:
model_path

WindowsPath('C:/Users/derrjohn/OneDrive - Qualcomm/Desktop/git/qnn_sample_apps/models/cpu-deepseek-r1-distill-qwen-7b/deepseek-r1-distill-qwen-7b-cpu-int4-rtn-block-32-acc-level-4.onnx')

In [10]:
session_options = ort.SessionOptions()

# Creating an inference session for the embedding graph
session = ort.InferenceSession(model_path)

session.get_providers()

['CPUExecutionProvider']

In [11]:
inputs = session.get_inputs()
outputs = session.get_outputs()
input_0 = inputs[0]
output_0 = outputs[0]

In [12]:
print(f"Expected Input Shape: {input_0.shape}")
print(f"Expected Input Type: {input_0.type}")
print(f"Expected Input Name: {input_0.name}")

Expected Input Shape: ['batch_size', 'sequence_length']
Expected Input Type: tensor(int64)
Expected Input Name: input_ids


In [13]:
print(f"Expected Output Shape: {output_0.shape}")
print(f"Expected Output Type: {output_0.type}")
print(f"Expected Output Name: {output_0.name}")

Expected Output Shape: ['batch_size', 'sequence_length', 152064]
Expected Output Type: tensor(float)
Expected Output Name: logits


In [14]:
for layer in inputs:
    print(f"Name: {layer.name}\n\tExpected Input Shape: {layer.shape}\n\tExpected Input Type: {layer.type}")
    print("*"*100)

Name: input_ids
	Expected Input Shape: ['batch_size', 'sequence_length']
	Expected Input Type: tensor(int64)
****************************************************************************************************
Name: attention_mask
	Expected Input Shape: ['batch_size', 'total_sequence_length']
	Expected Input Type: tensor(int64)
****************************************************************************************************
Name: past_key_values.0.key
	Expected Input Shape: ['batch_size', 4, 'past_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: past_key_values.0.value
	Expected Input Shape: ['batch_size', 4, 'past_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: past_key_values.1.key
	Expected Input Shape: ['batch_size', 4, 'past_sequence_length', 128]

In [15]:
for layer in outputs:
    print(f"Name: {layer.name}\n\tExpected Input Shape: {layer.shape}\n\tExpected Input Type: {layer.type}")
    print("*"*100)

Name: logits
	Expected Input Shape: ['batch_size', 'sequence_length', 152064]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.0.key
	Expected Input Shape: ['batch_size', 4, 'total_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.0.value
	Expected Input Shape: ['batch_size', 4, 'total_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.1.key
	Expected Input Shape: ['batch_size', 4, 'total_sequence_length', 128]
	Expected Input Type: tensor(float)
****************************************************************************************************
Name: present.1.value
	Expected Input Shape: ['batch_size', 4, 'total_sequence_length', 128]
	Expec

In [16]:
# Load in tokenizer using tokenizer path above
tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [17]:
query_bank = {"security":"<｜User｜>\nImagine you are a cyber security professional. Provide step by step reasons why AI models should be ran locally. Please consider all aspects of data privacy and cyber security\n<｜Assistant｜><think>\n",
              "cooking":"<｜User｜>\nGive me a step-by-step baked chicken recipe, including ingredients, cook time, and sides.\n<｜Assistant｜>\n",
              "therapist":"<｜User｜>\nImagine you are a therapist with a background in cyber security. I'm am currently very anxious about my data being stolen\
              can you provide me remedies to help with my depression and anxiety\n<｜Assistant｜><think>\n"
             }

In [18]:
# An initial query
init_query = query_bank["therapist"]
encoding = tokenizer.encode(init_query)

In [19]:
print("Token IDs:", encoding.ids)
print("Tokens:", encoding.tokens)

Token IDs: [151646, 151644, 198, 51057, 498, 525, 264, 41763, 448, 264, 4004, 304, 20847, 4763, 13, 358, 2776, 1079, 5023, 1602, 37000, 911, 847, 821, 1660, 22329, 1060, 646, 498, 3410, 752, 50136, 311, 1492, 448, 847, 18210, 323, 18056, 198, 151645, 151648, 198]
Tokens: ['<｜begin▁of▁sentence｜>', '<｜User｜>', 'Ċ', 'Imagine', 'Ġyou', 'Ġare', 'Ġa', 'Ġtherapist', 'Ġwith', 'Ġa', 'Ġbackground', 'Ġin', 'Ġcyber', 'Ġsecurity', '.', 'ĠI', "'m", 'Ġam', 'Ġcurrently', 'Ġvery', 'Ġanxious', 'Ġabout', 'Ġmy', 'Ġdata', 'Ġbeing', 'Ġstolen', 'ĠĠĠĠĠĠĠĠĠĠĠĠĠ', 'Ġcan', 'Ġyou', 'Ġprovide', 'Ġme', 'Ġremedies', 'Ġto', 'Ġhelp', 'Ġwith', 'Ġmy', 'Ġdepression', 'Ġand', 'Ġanxiety', 'Ċ', '<｜Assistant｜>', '<think>', 'Ċ']


In [20]:
input_ids = encoding.ids
input_ids

[151646,
 151644,
 198,
 51057,
 498,
 525,
 264,
 41763,
 448,
 264,
 4004,
 304,
 20847,
 4763,
 13,
 358,
 2776,
 1079,
 5023,
 1602,
 37000,
 911,
 847,
 821,
 1660,
 22329,
 1060,
 646,
 498,
 3410,
 752,
 50136,
 311,
 1492,
 448,
 847,
 18210,
 323,
 18056,
 198,
 151645,
 151648,
 198]

In [21]:
# Preparing inputs for prompt

# Number of input sequences processed simultaneously
batch_size = 1

# Current sequence length for initial prompt (number of tokens in current sequence)
seq_len = len(input_ids)#.shape[2]

# Dimensionality of each token embedding vector
# hidden_size = embedding_output.shape[2]

# Number of attention heads in each transformer layer
num_heads = 28

# Size of each attention head (should be hidden_size // num_heads
attn_head_size = 128 #hidden_size//num_heads # ex. 1536/12 = 128

# Total number of transformer layers
num_layers = 28

# This is not the model's global context window (131072), this is the max number of tokens passed in the first forward pass
max_seq_len = len(input_ids)

# Sampling temperature for softmax-based logit scaling
temp = 0.7

# Number of key/value heads (key/value heads are shared amongst attention heads)
num_key_value_heads = 4

In [22]:
len(input_ids)

43

In [23]:
# pad the inputs to expected size of seq_len of 64
# target_seq_len = 64
# input_ids += [pad_token_id] * (target_seq_len - len(input_ids))
input_ids = np.array([input_ids], dtype=np.int64)
input_ids.shape

(1, 43)

In [24]:
attention_mask = np.ones((batch_size, max_seq_len), dtype=np.int64)
attention_mask

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [25]:
attention_mask.shape

(1, 43)

In [26]:
# Let's initialize our KV cache for all transformer layers
empty_kv = {}
for i in range(num_layers):
    # Shape of key and value tensors for each transformer layer
    past_shape = (batch_size, num_key_value_heads, max_seq_len, attn_head_size)

    # Initialize past keys for layer i (used in attention mechanism to avoid recomputation
    empty_kv[f"past_key_values.{i}.key"] = np.zeros(past_shape, dtype=np.float32)

    # Initialize past values for layer i
    empty_kv[f"past_key_values.{i}.value"] = np.zeros(past_shape, dtype=np.float32)

len(empty_kv)

56

In [27]:
empty_kv.keys()

dict_keys(['past_key_values.0.key', 'past_key_values.0.value', 'past_key_values.1.key', 'past_key_values.1.value', 'past_key_values.2.key', 'past_key_values.2.value', 'past_key_values.3.key', 'past_key_values.3.value', 'past_key_values.4.key', 'past_key_values.4.value', 'past_key_values.5.key', 'past_key_values.5.value', 'past_key_values.6.key', 'past_key_values.6.value', 'past_key_values.7.key', 'past_key_values.7.value', 'past_key_values.8.key', 'past_key_values.8.value', 'past_key_values.9.key', 'past_key_values.9.value', 'past_key_values.10.key', 'past_key_values.10.value', 'past_key_values.11.key', 'past_key_values.11.value', 'past_key_values.12.key', 'past_key_values.12.value', 'past_key_values.13.key', 'past_key_values.13.value', 'past_key_values.14.key', 'past_key_values.14.value', 'past_key_values.15.key', 'past_key_values.15.value', 'past_key_values.16.key', 'past_key_values.16.value', 'past_key_values.17.key', 'past_key_values.17.value', 'past_key_values.18.key', 'past_key_v

In [28]:
init_prompt_inputs = {
    "input_ids": input_ids,
    "attention_mask":attention_mask,
    **empty_kv,
}
init_prompt_inputs

{'input_ids': array([[151646, 151644,    198,  51057,    498,    525,    264,  41763,
            448,    264,   4004,    304,  20847,   4763,     13,    358,
           2776,   1079,   5023,   1602,  37000,    911,    847,    821,
           1660,  22329,   1060,    646,    498,   3410,    752,  50136,
            311,   1492,    448,    847,  18210,    323,  18056,    198,
         151645, 151648,    198]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'past_key_values.0.key': array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
      

In [29]:
init_prompt_inputs.get("past_key_values.0.key").shape

(1, 4, 43, 128)

In [30]:
# Run embedding session first
session_output = session.run(None, init_prompt_inputs)
# print("Logits:\n(batch, sequence length, vocab size)")
session_output[0].shape

(1, 43, 152064)

In [31]:
print("Logits:\n(batch, sequence length, vocab size)")
session_output[0].shape

Logits:
(batch, sequence length, vocab size)


(1, 43, 152064)

In [32]:
print("KV Cache:\n(batch, num_kv_heads, sequence length, attn_head_size)")
session_output[1].shape

KV Cache:
(batch, num_kv_heads, sequence length, attn_head_size)


(1, 4, 43, 128)

### To get longer initial context run ctx session over multiple prompts BUT use updated key/values after each prompt

In [33]:
# Update kv cache
present_kv = {f"past_key_values.{i}.key": session_output[1 + i * 2] for i in range(num_layers)}
present_kv.update({f"past_key_values.{i}.value": session_output[1 + i * 2 + 1] for i in range(num_layers)})
present_kv

{'past_key_values.0.key': array([[[[-4.49215472e-01,  1.13862538e+00,  5.71835518e-01, ...,
           -1.24166397e+02, -1.70365997e+02, -1.21686340e+02],
          [ 2.88219571e+00,  1.15904176e+00,  9.66837585e-01, ...,
           -1.29335052e+02, -1.66706726e+02, -1.25295822e+02],
          [ 4.01041555e+00,  1.19298697e+00, -9.88663971e-01, ...,
           -1.23552437e+02, -1.70032654e+02, -1.22852341e+02],
          ...,
          [ 3.33651590e+00, -1.34508824e+00,  4.99407738e-01, ...,
           -1.22955055e+02, -1.70117859e+02, -1.22066917e+02],
          [ 1.57928839e-01, -1.39988327e+00,  1.42148876e+00, ...,
           -1.22963264e+02, -1.70106339e+02, -1.22569038e+02],
          [-2.97307158e+00, -1.18990171e+00,  1.84630477e+00, ...,
           -1.22709915e+02, -1.70635895e+02, -1.22028664e+02]],
 
         [[ 5.33364487e+00, -2.57349110e+00,  3.04576778e+00, ...,
           -4.18381119e+01, -1.35962601e+02,  1.10642708e+02],
          [ 4.72010183e+00, -6.78979874e-01,  2

In [34]:
present_kv.keys()

dict_keys(['past_key_values.0.key', 'past_key_values.1.key', 'past_key_values.2.key', 'past_key_values.3.key', 'past_key_values.4.key', 'past_key_values.5.key', 'past_key_values.6.key', 'past_key_values.7.key', 'past_key_values.8.key', 'past_key_values.9.key', 'past_key_values.10.key', 'past_key_values.11.key', 'past_key_values.12.key', 'past_key_values.13.key', 'past_key_values.14.key', 'past_key_values.15.key', 'past_key_values.16.key', 'past_key_values.17.key', 'past_key_values.18.key', 'past_key_values.19.key', 'past_key_values.20.key', 'past_key_values.21.key', 'past_key_values.22.key', 'past_key_values.23.key', 'past_key_values.24.key', 'past_key_values.25.key', 'past_key_values.26.key', 'past_key_values.27.key', 'past_key_values.0.value', 'past_key_values.1.value', 'past_key_values.2.value', 'past_key_values.3.value', 'past_key_values.4.value', 'past_key_values.5.value', 'past_key_values.6.value', 'past_key_values.7.value', 'past_key_values.8.value', 'past_key_values.9.value', '

In [35]:
# Dimension checks
present_kv["past_key_values.0.key"].shape

(1, 4, 43, 128)

In [36]:
present_kv["past_key_values.27.value"].shape

(1, 4, 43, 128)

In [37]:
logits = session_output[0]
logits

array([[[ 8.9006424e-03,  1.4297953e+00,  6.2274075e-01, ...,
         -2.6108739e+00, -2.6119459e+00, -2.6112013e+00],
        [-2.1791601e-01,  8.0655575e-01,  3.7205505e-01, ...,
         -2.4572210e+00, -2.4578846e+00, -2.4573443e+00],
        [ 2.1729212e+00,  3.4739676e+00,  9.5246315e-02, ...,
         -6.2419748e+00, -6.2421360e+00, -6.2440376e+00],
        ...,
        [ 3.4915257e-01, -3.0515790e+00,  3.9636135e-01, ...,
          1.4603293e+00,  1.4665092e+00,  1.4544331e+00],
        [ 7.6523833e+00,  2.4325323e+00,  5.5764885e+00, ...,
          2.0563369e+00,  2.0597391e+00,  2.0575371e+00],
        [ 5.5887308e+00,  8.2564764e+00,  1.1005019e+01, ...,
          5.1189537e+00,  5.1198564e+00,  5.1188397e+00]]],
      shape=(1, 43, 152064), dtype=float32)

In [38]:
logits[0,-1].shape

(152064,)

In [39]:
def softmax_numpy(x: np.array, temperature: float=1) -> np.array:
    # stabilize x in case of large numbers 
    x = x - np.max(x)

    # Apply temperature
    x = x/temperature

    # Apply Softmax
    return np.exp(x)/np.sum(np.exp(x), axis=-1)

def top_k_probas(probas: np.array, k: int=5) -> np.array:
    # Copy probas so in-place operations don't work on original variable
    probas = probas.copy()
    # Normalize probabilities
    probas /= np.sum(probas)
    # Using -probas to get in descending order
    top_indices_sorted = np.argsort(-probas)[:k]
    top_k_probas = probas[top_indices_sorted]

    # Renormalize top-k probabilites to sum to 1 (probabilites must sum to 1 to use np.random.choice
    top_k_probas /= np.sum(top_k_probas)

    # Return top k probabilities
    return top_indices_sorted, top_k_probas

def apply_repetition_penalty(logits, generated_ids, penalty=1.1):
    for token_id in set(generated_ids):
        logits[token_id] /= penalty
    return logits

In [40]:
# Softmax implemented
# x-np.max(x) => for stability in case of large numbers
softmax = lambda x, temperature=1: np.exp((x-np.max(x))/temperature)/np.sum(np.exp((x-np.max(x))/temperature), axis=-1)

In [41]:
softmax_numpy(logits[0,-1])

array([1.1275379e-11, 1.6244918e-10, 2.5374323e-09, ..., 7.0487019e-12,
       7.0550640e-12, 7.0478957e-12], shape=(152064,), dtype=float32)

In [42]:
softmax(logits[0,-1])

array([1.1275379e-11, 1.6244918e-10, 2.5374323e-09, ..., 7.0487019e-12,
       7.0550640e-12, 7.0478957e-12], shape=(152064,), dtype=float32)

In [43]:
# Grabs last tokens logits
temp = 0.6
probas = softmax(logits[0,-1], temperature=temp)
# probas = probas / probas.sum()
next_token_id = int(np.random.choice(len(probas), p=probas)) #int(np.argmax(probas))
next_token_id

32313

In [44]:
np.sum(probas)

np.float32(0.99999994)

In [45]:
tokenizer.decode([next_token_id])

'Okay'

In [46]:
logits.shape

(1, 43, 152064)

In [49]:
# temp = 0.6
start = time.time()
max_tokens = 1000
top_k = 5
generated_ids = [next_token_id]
prev_seq_len = logits.shape[1]
# print(prev_seq_len)
# print(attention_mask.shape)
print("\nInitial Query:\n", init_query)
print("Generated:")
for _ in range(max_tokens):
    input_ids = np.array([[next_token_id]], dtype=np.int64)
    # print(tokenizer.decode(generated_ids, skip_special_tokens=True))
    print(tokenizer.decode([next_token_id], skip_special_tokens=True),end="")
    
    iter_inputs = {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    **present_kv,
    }

    session_output = session.run(None, iter_inputs)
    prev_seq_len += 1
    # Update attention mask
    attention_mask = np.ones((batch_size, prev_seq_len), dtype=np.int64)
    # Update KV Cache
    present_kv = {f"past_key_values.{i}.key": session_output[1 + i * 2] for i in range(num_layers)}
    present_kv.update({f"past_key_values.{i}.value": session_output[1 + i * 2 + 1] for i in range(num_layers)})
    # print(prev_seq_len)
    # print(present_kv.get("past_key_values.0.key").shape)
    # print(len(attention_mask))
    logits = session_output[0]

    token_logits = logits[0,-1]
    token_logits = apply_repetition_penalty(token_logits, generated_ids, penalty=1.1)
#     # Get probabilities
    probas = softmax(token_logits, temperature=temp)
    top_indices, top_probas = top_k_probas(probas, k=top_k) 
    next_token_id = int(np.random.choice(top_indices, p=top_probas)) #int(np.argmax(probas))
    generated_ids.append(next_token_id)


    if next_token_id == tokenizer.token_to_id("< | end_of_sentence | >"):
        break
end = time.time()
elapsed = end - start
tps = np.round((max_tokens / elapsed), 2)
print(f"\nTokens Per Second: {tps}")
output_text = tokenizer.decode(generated_ids, skip_special_tokens=True)



Initial Query:
 <｜User｜>
Imagine you are a therapist with a background in cyber security. I'm am currently very anxious about my data being stolen              can you provide me remedies to help with my depression and anxiety
<｜Assistant｜><think>

Generated:
Okay, so I'm really anxious about my data being stolen. That sounds pretty serious. I guess the first thing I should do is figure out why I'm feeling this way. Maybe it's because I take everything very seriously and don't like the idea of something important getting into the wrong hands.

I remember reading somewhere that anxiety can be linked to security concerns. So, if I'm worried about data theft, maybe I need to address both my anxiety and the underlying issues causing these fears. But how?

First, I think I should talk to someone about it. Maybe a therapist who specializes in cyber security? They might have strategies tailored for people with similar issues. Or perhaps a counselor who understands both mental health and tech